In [1]:
import numpy as np
import numpy.linalg as la
from numpy.random import seed, choice
import scipy
import scipy.linalg as sla
import scipy.stats
from scipy.stats import multivariate_normal
from two_stages_estimate import two_stage_estimation
import rpy2.robjects as R
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

seed(22032018)

########################################################
################## Simulate dataset ####################
########################################################
time = 100 # time length
n = 3 # number of test
nc = 12 # number of controls to be generated
n_cntl = 4 # number of controls to be used for each response
impact_begin = 80

# generate control datasets
stats = importr("stats")
cntl_data_pool = np.zeros((time, nc))
for i in range(nc):
    cntl_data_pool[:, i] = np.array(stats.arima_sim([[0.6]], n=time))

cntl_data_pool = cntl_data_pool + np.abs(cntl_data_pool.min())
# force observations to be positive

# generate test datasets
test_data = np.zeros((time, n))
A = np.zeros((n, n)) # generate empty matrix, use to create AR(1) correlation
# matrix
graph_structure = sla.toeplitz(np.c_[1, 1, np.zeros(n-2)])
print(graph_structure)
Sigma_inv = sla.toeplitz(np.c_[10, 5, np.zeros(n-2)])
Sigma = la.inv(Sigma_inv)
mu = np.zeros((n, time))
tau = np.zeros((n, time))
for t in range(time):
    if (t == 0):
        mu[:, 0] = 1
    else:
        mu[:, t] = 0.8 * mu[:, t-1] + 0.1 * multivariate_normal.rvs(size=n)
    test_data[t, :] = mu[:, t] + cntl_data_pool[t, :n*2:2] \
                      + 2 * cntl_data_pool[t, 1:n*2:2] \
                      + multivariate_normal(mean=np.zeros(n), cov=Sigma).rvs()
  # add seasonality

xx = 2*np.pi/7*np.arange(time).reshape(-1, 1)
test_data += 0.1 * np.cos(xx) + 0.1 * np.sin(xx)

test_data = (test_data.T + np.abs(test_data.min(axis=0)).reshape(-1, 1) +1).T
# simulate causal impact
causal_period = np.arange(impact_begin, time) # campaign runs 20 periods

# simulate causal impact
for i in range(n):
    test_data[causal_period, i] = test_data[causal_period, i] \
                                  + 0.5*(i-1)*np.log(np.arange(time-impact_begin))

########################################################
################# Reorganize dataset ###################
########################################################
a = np.arange(2*n, nc)
cntl_data = np.empty((cntl_data_pool.shape[0], 0))
for i in range(0, n):
    cntl_data_select = cntl_data_pool[:, choice(a, n_cntl-2)]
    cntl_data = np.hstack((cntl_data, cntl_data_pool[:, 2*i:2*i+2], cntl_data_select))

########################################################
##################### Fit model ########################
########################################################
## Stage 1:
# EMVS for estimating beta
iterloop = 10000
stationary = True
nseasons = 7
graph = True
burnin = 2000
num_sim_data = 10
cntl_index = np.ones(n, dtype=np.int32) * n_cntl

checked_index_test = (~np.isnan(test_data).any(axis=1)) + (~np.isinf(test_data).any(axis=1))
checked_index_cntl = (~np.isnan(cntl_data).any(axis=1)) + (~np.isinf(cntl_data).any(axis=1))
checked_index = checked_index_test * checked_index_cntl

n_bad_before_impact = impact_begin - sum(checked_index[:impact_begin])
n_bad_after_impact = time - impact_begin - sum(checked_index[impact_begin:])

impact_begin -= n_bad_before_impact
time -= n_bad_before_impact + n_bad_after_impact

test_data = test_data[checked_index,:]
cntl_data = cntl_data[checked_index,:]

causal_period = np.arange(impact_begin, time)

MultivariateCausalInferenceRes = \
  two_stage_estimation(test_data, cntl_index, cntl_data, 
                       graph = graph, graph_structure = graph_structure, 
                       circle = nseasons, causal_period = causal_period, 
                       s = 0.1,
                       emvs_iteration = 20, 
                       v0_value = np.linspace(1e-6, 0.02, 5),
                       mcmc_iterloop = iterloop, burnin = burnin, 
                       stationary = stationary, 
                       misspecification = False,
                       num_sim_data = num_sim_data, 
                       seed = 1, probs = 0.95,
                       plot_title = "EMVS plot")

# collect results
beta_hat = MultivariateCausalInferenceRes["beta hat"]
mcmc_output = MultivariateCausalInferenceRes["mcmc output"]
threshold = MultivariateCausalInferenceRes["threshold"]
ks_test_cntl = MultivariateCausalInferenceRes["ks test cntl"]

/home/daria/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


[[ 1.  1.  0.]
 [ 1.  1.  1.]
 [ 0.  1.  1.]]
Starting Bayesian EM variable selection...


/home/daria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in log
/home/daria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in multiply
/home/daria/Documents/SkoltechHSE/ML/Project/Final/estimate_counterfactual.py:45: RuntimeWarning: invalid value encountered in log
  2*np.log(np.divide(theta,(1-theta)) + 1e-10)) / (np.divide(1,v1)-np.divide(1,v0_value)))
/home/daria/Documents/SkoltechHSE/ML/Project/Final/estimate_counterfactual.py:62: RuntimeWarning: invalid value encountered in less
  beta_hat[np.abs(beta_hat) < beta_star] = 0
MCMC sampling:  15%|█▌        | 15/100 [00:01<00:05, 14.41it/s]


LinAlgError: singular matrix